# Chroma Authentication

This tutorial aims to explain how authentication can be setup in Chroma.

> **Important**: The concept of authentication is only applicable to Client/Server deployments. If you are using Chroma in a standalone mode, authentication is not applicable.

## Concepts

### Architecture Overview

![Authentication Architecture](assets/auth-architecture.png "Authentication Architecture")

### Authentication Flow (Sequence)

The authentication sequence is applied for every request. It is important to understand that credential computation or retrieval (e.g. from external auth providers) is only done once for the first authenticated request. Subsequent requests will use the same credentials.

The authentication flow is as follows:

![Authentication Flow](assets/auh-sequence.png "Authentication Flow")

### Preemptive Authentication

In its current release the authentication in Chroma works in a preemptive mode. This means that the client is responsible for sending the authentication information on every request. The server will not challenge the client for authentication.

> **Warning**: There are security risks involved with preemptive authentication in that the client might unintentionally send credentials to malicious or unintended server. When deploying authentication users are encouraged to use HTTPS (always verify server certs), to use secure providers (e.g. JWT) 
> and apply good security practices.

### Authentication Provider

Authentication in Chroma is handled by Authentication Providers. Providers are pluggable modules that allow Chroma to abstract the authentication mechanism from the rest of the system.

Chroma ships with the following build-in providers:
- Basic Authentication
- JWT Authentication (work in progress)

### Client-side Authentication

Client-side authentication refers to the process of preparing and communicating credentials information on the client-side and sending that information the Chroma server.

### Server-side Authentication

Server-side authentication refers to the process of validating the credentials information received from the client and authenticating the client.


## Configuration

### Server Configuration

The following configuration parameters are available for the server (defined as environment variables):

- `CHROMA_SERVER_AUTH_PROVIDER` - It indicates the authentication provider class to use. In this case we are using the `chromadb.auth.BasicAuthServerProvider` class.
- `CHROMA_SERVER_AUTH_PROVIDER_CONFIG` - It is a dictionary that contains the configuration parameters for the authentication provider.

#### Basic Authentication Provider Configuration

The following configuration parameters are available for the Basic Authentication Provider (defined as environment variables):

- `CHROMA_SERVER_AUTH_BASIC_USERNAME` - The username to use for authentication.
- `CHROMA_SERVER_AUTH_BASIC_PASSWORD` - The password to use for authentication.
- `CHROMA_SERVER_AUTH_IGNORE_PATHS` - A comma separated list of paths to ignore for authentication. This is useful for public endpoints that do not require authentication. This defaults to `/api/v1, /api/v1/heartbeat, /api/v1/version`.

When using `CHROMA_SERVER_AUTH_PROVIDER_CONFIG` to define the basic auth configuration, the following formats are available:

- JSON format: `{"username":"admin","password":"admin"}`
- File path: `/path/to/htpasswd/file` where the file is in htpasswd format (user:password) - single line with plaintext password.

### Client Configuration

The following configuration parameters are available for the client (defined as environment variables or passed as configuration when setting up the client):

- `CHROMA_CLIENT_AUTH_PROVIDER` - It indicates the authentication provider class to use. In this case we are using the `chromadb.auth.BasicAuthClientProvider` class.
- `CHROMA_CLIENT_AUTH_PROVIDER_CONFIG` - It is a dictionary that contains the configuration parameters for the authentication provider. This can also be a file path with htpasswd format (user:password) - single line with plaintext password.

#### Basic Authentication Provider Configuration

The following configuration parameters are available for the Basic Authentication Provider (defined as environment variables or passed as configuration when setting up the client):

- `CHROMA_CLIENT_AUTH_BASIC_USERNAME` - The username to use for authentication.
- `CHROMA_CLIENT_AUTH_BASIC_PASSWORD` - The password to use for authentication.

When using `CHROMA_CLIENT_AUTH_PROVIDER_CONFIG` to define the basic auth configuration, the following formats are available:

- JSON format: `{"username":"admin","password":"admin"}`
- File path: `/path/to/htpasswd/file` where the file is in htpasswd format (user:password) - single line with plaintext password.

## Setting Up

### Before You Begin

Make sure you have either `chromadb` or `chromadb-client` installed. You can do that by running the following command:

```bash
pip install chromadb
```
or

```bash
pip install chromadb-client
```

Make sure Chroma Server is running. Use one of the following methods to start the server:

From the command line:

> Note: The below options will configure the server to use Basic Authentication with the username `admin` and password `admin`.

```bash
export CHROMA_USER=admin
export CHROMA_PASSWORD=admin
docker run --rm --entrypoint htpasswd httpd:2 -Bbn ${CHROMA_USER} ${CHROMA_PASSWORD} > server.htpasswd
CHROMA_SERVER_AUTH_CREDENTIALS_FILE="./server.htpasswd" \
CHROMA_SERVER_AUTH_CREDENTIALS_PROVIDER='chromadb.auth.HtpasswdServerAuthCredentialsProvider' \
CHROMA_SERVER_AUTH_PROVIDER='chromadb.auth.basic.BasicAuthServerProvider' \
uvicorn chromadb.app:app --workers 1 --host 0.0.0.0 --port 8000  --proxy-headers --log-config log_config.yml
```

With Docker Compose:

> Note: You need to clone the git repository first and run the command from the repository root.

```bash
export CHROMA_USER=admin
export CHROMA_PASSWORD=admin
docker run --rm --entrypoint htpasswd httpd:2 -Bbn ${CHROMA_USER} ${CHROMA_PASSWORD} > server.htpasswd
cat << EOF > .env
CHROMA_SERVER_AUTH_CREDENTIALS_FILE="/chroma/server.htpasswd"
CHROMA_SERVER_AUTH_CREDENTIALS_PROVIDER='chromadb.auth.HtpasswdServerAuthCredentialsProvider'
CHROMA_SERVER_AUTH_PROVIDER='chromadb.auth.basic.BasicAuthServerProvider'
EOF
docker-compose up -d --build 
```


In the snippet below we will configure the client to use Basic Authentication.

Configuration parameters:

- `chroma_client_auth_provider` - It indicates the authentication provider class to use. In this case we are using the `chromadb.auth.BasicAuthClientProvider` class.
- `chroma_client_auth_provider_config` - It is a dictionary that contains the configuration parameters for the authentication provider. In this case we are using the `username` and `password` parameters in a JSON format.

### Basic Authentication

In [2]:
import chromadb
from chromadb import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:admin"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint

client.get_version()  # this should work with or without authentication - it is a public endpoint

client.list_collections()  # this is a protected endpoint and requires authentication



[]

#### Verifying Authentication (Negative Test)

In [3]:
# Try to access a protected endpoint without authentication
import sys

client = chromadb.HttpClient()
try:
    client.list_collections()
except Exception as e:
    if "Unauthorized" in str(e):
        print("As expected, you are not authorized to access protected endpoints.", file=sys.stderr)
    else:
        raise e

As expected, you are not authorized to access protected endpoints.
